# Feature reduction

We reduce 4608 features to 512 features learned by unsupervised UMAP algorithm. 

This requires T5, esm and protBERT embeddings (see INSTALL.md for downloading instructions)

In [1]:
import numpy as np
import os
import umap

/home/joni/miniconda3/envs/cafa5/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/joni/miniconda3/envs/cafa5/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/joni/miniconda3/envs/cafa5/lib/python3.9/site-packages/umap/distances.

In [2]:
def load_all_embeddings():
    train_protein_ids = np.load('../input/t5embeds/train_ids.npy')    
    test_protein_ids = np.load('../input/t5embeds/test_ids.npy')
    train_embeddings = np.concatenate([
        np.load('../input/t5embeds/train_embeds.npy'),
        np.load('../input/esm23b/train_embeds_esm2_t36_3B_UR50D.npy'),
        np.load('../input/protbert/train_embeddings.npy')
    ], axis=1)
    
    test_embeddings = np.concatenate([
        np.load('../input/t5embeds/test_embeds.npy'),
        np.load('../input/esm23b/test_embeds_esm2_t36_3B_UR50D.npy'),
        np.load('../input/protbert/test_embeddings.npy')
    ], axis=1)
    return (train_protein_ids, train_embeddings, test_protein_ids, test_embeddings)

train_protein_ids, train_embeddings, test_protein_ids, test_embeddings = load_all_embeddings()

In [6]:
train_embeddings.shape

(142246, 4608)

In [3]:
# normalize the embeddings

from sklearn.preprocessing import StandardScaler

In [4]:
# Initialize a scaler
scaler = StandardScaler()

# Fit on training set only
scaler.fit(train_embeddings)

# Apply transform to both the training set and the test set
train_embeddings = scaler.transform(train_embeddings)
test_embeddings = scaler.transform(test_embeddings)

In [5]:
# use umap to reduce the number of features
reducer = umap.UMAP(
    n_neighbors=30,
    n_components=512,
    metric='euclidean'
)

In [6]:
reducer = reducer.fit(train_embeddings)

In [7]:
test_embeddings_umap = reducer.transform(test_embeddings)

In [8]:
reducer.embedding_.shape

(142246, 512)

In [9]:
test_embeddings_umap.shape

(141865, 512)

In [10]:
np.save('../input/train_emb_umap_512.npy', reducer.embedding_)
np.save('../input/test_emb_umap_512.npy', test_embeddings_umap)